In [1]:
import os
import glob
import re
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import clear_output

import mbuild as mb

import warnings
warnings.filterwarnings('ignore')

In [45]:
# Read in and create a dictionary of all the compounds
files = glob.glob('./molecules-mol2/foyer_molecules/ante_types/*.mol2')
compounds = {'Select your molecule': None}
for file in files:
    molname = re.sub('.mol2','',re.sub('^.*/','',file))
    compounds[molname] = mb.load(file, use_parmed=True)

In [46]:
# Define some globale variable
# These variables will be update when the widget event handlers are called
COMPOUND = None
BOX_OF_COMPOUNDS = None



def visualize(compound):
    # A method to filter out numeric string in particle name
    # before being visualize by py3dmol (so the color scheme
    # could be consistent)
    vis_compound = mb.clone(compound)
    for particle in vis_compound.particles():
        particle.name = ''.join(filter(str.isalpha, 
                                    particle.name))
        
    display(vis_compound.visualize())
    
def compound_handler(obj):
    global COMPOUND 
    COMPOUND = compound_dropdown.value
    with out1: 
        clear_output()
        if compound_dropdown.value:
            visualize(compound_dropdown.value)
            


def box_handler(obj):
    box = [box_slider.value] * 3
    n_compounds = n_slider.value
    compound = compound_dropdown.value
    
    box_of_compound = mb.fill_box(compound=compound, 
                                  n_compounds=n_compounds,
                                  box=box)
    global BOX_OF_COMPOUNDS
    BOX_OF_COMPOUNDS = box_of_compound
    with out2:
        clear_output()
        visualize(box_of_compound)
  

In [47]:
compound_dropdown = widgets.Dropdown(options=compounds)
display(compound_dropdown)

out1 = widgets.Output()
display(out1)

compound_dropdown.observe(compound_handler, names='value')
compound_dropdown.observe(box_handler, names='value')


Dropdown(options={'Select your molecule': None, 'methyl-acetate': <Compound 11 particles, non-periodic, 10 bon…

Output()

In [44]:
# We will create some orthogonal box from the box handler
box_slider = widgets.FloatSlider(min=0, max=10, description='Box size', value=5)
n_slider = widgets.IntSlider(min=1, max=100, description='n molecules', value=50)

display(box_slider)
display(n_slider)

out2 = widgets.Output()
display(out2)

box_slider.observe(box_handler, names='value')
n_slider.observe(box_handler, names='value')

FloatSlider(value=5.0, description='Box size', max=10.0)

IntSlider(value=50, description='n molecules', min=1)

Output()

In [68]:
# Experiment with SMILES string
def smiles_handler(obj):
    try:
        compound = mb.load(smiles_box.value, smiles=True)
    except:
        compound = None
    with out3:
        clear_output()
        if compound:
            visualize(compound)
        else:
            print('Invalid SMILES string')
        
        
smiles_box = widgets.Text(
    placeholder='Enter a SMILES string',
    description='SMILES:',
    disabled=False
)

display(smiles_box)

out3 = widgets.Output()
display(out3)

smiles_box.observe(smiles_handler, names='value')

Text(value='', description='SMILES:', placeholder='Enter a SMILES string')

Output()